In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
%cd /content/drive/MyDrive/ML2021

In [ ]:
import pandas as pd
import numpy as np
#import missingno as msno
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from tensorflow import keras
from keras import layers
from keras.models import Sequential, Model
from tensorflow.keras import optimizers
import random
from scipy.ndimage import gaussian_filter1d

In [ ]:
data=pd.read_csv('wtbdata_245days.csv')
data

In [ ]:
data=data[data['Day']>=36].reset_index()
data=data.drop(['index','TurbID','Day','Tmstamp'],axis=1)
data

In [ ]:
columns=['Wspd','Wdir','Etmp','Itmp','Ndir','Pab1','Pab2','Pab3','Prtv','Patv']
for column in columns:
  data[column].fillna(method='bfill', inplace=True)

todelete=[]
todelete.extend(list(data[(data['Wdir']>180) | (data['Wdir']<-180)].index))
todelete.extend(list(data[(data['Ndir']>720) | (data['Ndir']<-720)].index))
todelete.extend(list(data[(data['Patv']<0) & (data['Wspd']>2.5)].index))
todelete.extend(list(data[(data['Pab1']>89) | (data['Pab2']>89) | (data['Pab3']>89)].index))
todelete.extend(list(data[(data['Patv']<0)].index))

data['Patv'].iloc[pd.unique(todelete)]=0.0
ana=data['Patv']

In [ ]:
time=[0 for i in range(144*210)]
for i in range(134):
  time=np.sum([time,ana[i*144*210:(i+1)*144*210].tolist()],axis=0).tolist()

In [ ]:
#single_move_avg=gaussian_filter1d(data['Patv'],5)
farm_move_avg=gaussian_filter1d(time,10)

In [ ]:
for i in range(134):
  temp=data['Patv'][i*144*210:(i+1)*144*210]
  data['Patv'][i*144*210:(i+1)*144*210]=gaussian_filter1d(temp,10)

In [ ]:
for i in range(134):
  temp=data['Patv'][i*144*210:(i+1)*144*210]
  temp_fil=gaussian_filter1d(temp,5)
  data['Patv'][i*144*210:(i+1)*144*210]=temp_fil

In [ ]:
def generator_val(num_turbine,data,label,lookback,delay,min_index,max_index,step,batch_size,shuffle=False):

  i=min_index+lookback

  rows=np.arange(i,i+batch_size)
 
  samples=np.zeros((len(rows),lookback//step,len(data[0]),num_turbine))
  targets=np.zeros((len(rows),))

  for j,row in enumerate(rows):
    start=rows[j]-lookback
    end=rows[j]

    turbines=np.zeros((lookback//step,len(data[0]),num_turbine))
    power_sum=0
    power_sum=label[end+delay]

    #turbine_list=[124,123,102,101,80,79,13,12,36,35,58,57]
    turbine_list=[124,102,80,13,36,58]

    for k in range(len(turbine_list)):
      new_start=start+(turbine_list[k]-1)*144*210
      new_end=end+(turbine_list[k]-1)*144*210
      
      indices=range(new_start,new_end,step)
      turbines[:,:,k]=data[indices]
    
    samples[j]=turbines
    targets[j]=power_sum

  return samples,targets

In [ ]:
x_scaler = MinMaxScaler()
data=x_scaler.fit_transform(data.values)

In [ ]:
y_scaler = MinMaxScaler()
label = y_scaler.fit_transform(np.array(farm_move_avg).reshape(-1,1))

In [ ]:
lookback=12*144
step=6
delay=2*144
num_turbine=6
time_slice=lookback//step


train_gen=generator_val(num_turbine,
           data,
           label,
           lookback=lookback,
           delay=delay,
           min_index=0,
           max_index=29663,
           step=step,
           batch_size=27936)

val_gen=generator_val(num_turbine,
           data,
           label,
           lookback=lookback,
           delay=delay,
           min_index=25920,
           max_index=27935,
           step=step,
           batch_size=288)

test_gen=generator_val(num_turbine,
           data,
           label,
           lookback=lookback,
           delay=delay,
           min_index=27936,
           max_index=29951,
           step=step,
           batch_size=288)

In [ ]:
input = layers.Input(shape=(time_slice,10,num_turbine))
x = layers.Conv2D(12,(1,1))(input)
x = layers.BatchNormalization()(x)
x = layers.Activation('relu')(x)

x = layers.Conv2D(24,(1,1))(x)
x = layers.BatchNormalization()(x)
x = layers.Activation('relu')(x)

x = layers.Conv2D(1,(1,1))(x)
x = layers.BatchNormalization()(x)
x = layers.Activation('relu')(x)
x = layers.Reshape((time_slice,len(data[0])))(x)

x = layers.Conv1D(32,4)(x)
x = layers.BatchNormalization()(x)
x = layers.Activation('relu')(x)
x = layers.AveragePooling1D(pool_size=2)(x)

x = layers.Conv1D(64,4)(x)
x = layers.BatchNormalization()(x)
x = layers.Activation('relu')(x)
x = layers.AveragePooling1D(pool_size=2)(x)

x = layers.Conv1D(128,4)(x)
x = layers.BatchNormalization()(x)
x = layers.Activation('relu')(x)

x = layers.LSTM(128, return_sequences=True, dropout=0.2, recurrent_dropout=0.2)(x)
x = layers.LSTM(64,return_sequences=False,dropout=0.2)(x)
output = layers.Dense(1)(x)
model=Model(input,output)
model.summary()

In [ ]:
model.compile(optimizer=optimizers.Adam(learning_rate=0.001), loss="mse", metrics=["mae",'mse'])
history = model.fit(train_gen[0],train_gen[1],batch_size=128,epochs=20)
model.save('cconv_batchnomalization1.h5')

In [ ]:
mae=history.history['mae']
val_mae=history.history['val_mae']
loss=history.history['loss']
val_loss=history.history['val_loss']
epochs=range(1,len(loss)+1)

plt.figure()
plt.plot(epochs,mae,'bo',label='Taining mae')
plt.plot(epochs,val_mae,'b',label='validation mae')
plt.title=('Training and validation mae')
plt.legend()

plt.figure()
plt.plot(epochs,loss,'bo',label='Taining loss')
plt.plot(epochs,val_loss,'b',label='validation loss')
plt.title=('Training and validation loss')
plt.legend()


plt.show()

In [ ]:
result=model.evaluate(test_gen[0],test_gen[1])

In [ ]:
prediction=model.predict(test_gen[0])

In [ ]:
c=model.predict(train_gen[0],batch_size=256)

In [ ]:
plt.figure(figsize=(15,5))
plt.plot(range(144*10),c[:10*144],'r',label='train_pre')
plt.plot(range(144*10),label[2016:2016+144*10],'b',label='true')
#plt.plot(range(1440,1440+288,1),label[-288:],'b')
#plt.plot(range(1440,1440+288,1),prediction,'g',label='test_pre')
plt.legend()

plt.figure(figsize=(15,5))
plt.plot(range(144*10),c[-144*10:],'r',label='train_pre')
plt.plot(range(144*10),label[28512:29952],'b',label='true')
plt.plot(range(1440,1440+288,1),label[-288:],'b')
plt.plot(range(1440,1440+288,1),prediction,'g',label='test_pre')
plt.legend()
plt.show()